In [2]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [11]:
query = 'SELECT rede,sigla_uf,  id_municipio, sum(alunos_soma) as alunos_soma, sum(professores_soma) as professores_soma FROM `repositoriodedadosgpsp.Datalake.INEP_media_aluno_docente` group by rede,sigla_uf,  id_municipio'

In [4]:
client = bigquery.Client()

In [12]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 14987/14987 [00:01<00:00, 10772.87rows/s]


In [13]:
df

,rede,sigla_uf,id_municipio,alunos_soma,professores_soma
0,federal,AP,1600303,55711,2226
1,federal,DF,5300108,353549,17334
2,federal,RR,1400209,28864,1589
3,federal,AP,1600535,10444,618
4,federal,AC,1200401,56993,2378
...,...,...,...,...,...
14982,municipal,SP,3541109,37,3
14983,municipal,SP,3513850,231,14
14984,municipal,GO,5211602,59,6
14985,municipal,SC,4215901,52,2


In [10]:
sum(df['professores_soma'])

83163369

In [7]:
## antigo

sum(df['professores_soma'])

83849604

In [38]:
## Subindo para o GBQ

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14987 entries, 0 to 14986
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rede              14987 non-null  object
 1   sigla_uf          14987 non-null  object
 2   id_municipio      14987 non-null  object
 3   alunos_soma       14987 non-null  Int64 
 4   professores_soma  14987 non-null  Int64 
dtypes: Int64(2), object(3)
memory usage: 614.8+ KB


In [15]:


schema=[bigquery.SchemaField('rede','STRING',description='Rede'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('id_municipio','STRING',description='ID do município no IBGE'),
 bigquery.SchemaField('alunos_soma','INTEGER',description='Soma de aluno'),
 bigquery.SchemaField('professores_soma','INTEGER',description='Soma de aluno')
 ]

In [16]:
dataset_ref = client.dataset('perfil_remuneracao')
table_ref = dataset_ref.table('INEP_media_aluno_docente_rede')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=fb32e835-1156-4d8a-b5e5-e00a193806ea>